In [4]:
import qmps
import xmps
import cirq
from xmps import iMPS
import numpy as np
def ap(A, _abs = True):
    if _abs:
        print(np.around(np.abs(A), 2))
    else:
        print(np.around(A, 2))

In [23]:
from numpy.linalg import svd

def S(ρ):
    # S = Σᵢρᵢlog(ρᵢ) where ρᵢ = |λᵢ|^2 and λᵢ = singular values 
    u, s, v = svd(ρ) 
    s = np.square(np.abs(s)) 
    return np.dot(np.log2(s), s)
    

### Alternating Hamiltonians to get 3 state evolution (didn't work)

In [66]:
from qmps.represent import FullStateTensor, FullEnvironment, Environment, ShallowFullStateTensor, Tensor
from qmps.tools import unitary_to_tensor, environment_from_unitary, tensor_to_unitary, get_env_exact
from xmps.spin import U4
from xmps.iMPS import Map, iMPS
from qmps.time_evolve_tools import merge, put_env_on_left_site, put_env_on_right_site
from tqdm.notebook import tqdm
from scipy.linalg import expm
from scipy.optimize import minimize
import numpy as np
import cirq

def two_site_sim(params, U1, U2,  W, A_input1, A_input2):
    
    """
    Simulate a quantum state of the form:
    
    |    |    |    |    |----|    |
    |    |    |    |-U1-|    |    |
    |    |    |-U2-|    |    |    |
    |    |-U1-|    |    |    |    |
    |-L--|    |----W----|    |--R-|
    |    |U1'-|    |    |    |    |
    |    |    |U2'-|    |    |    |
    |    |    |    |U1'-|    |    |
    |    |    |    |    |----|    |
    
    and return the overlap of these wo states to find parameterisations of U1' and U2' that minimize this overlap.
    
    Inputs: 
    1) params: 2 * N where N is the number of parameters used to specify the unitary U1(2)
    2) U1/U2: A parameterized unitary that decomposes to native cirq gates
    3) W: exponentiated hamiltonian 
    
    Ouputs:
    1) Final state |ψ>
    """
    
    U1_ = ShallowFullStateTensor(2, params[:15])
    U2_ = ShallowFullStateTensor(2, params[15:])
    
    A1_ = unitary_to_tensor(cirq.unitary(U1_))
    A2_ = unitary_to_tensor(cirq.unitary(U2_))

    if A_input1 is None:
        A1 = unitary_to_tensor(cirq.unitary(U1))
        A2 = unitary_to_tensor(cirq.unitary(U2))
    else:
        A1 = A_input1
        A2 = A_input2
    
    A12 = merge(A1, A2)
    A12_ = merge(A1_, A2_)
    
    _, r = Map(A12,A12_).right_fixed_point()
    
    R = Environment(put_env_on_left_site(r), 'θR')
    left = put_env_on_right_site(r.conj().T)
    L = Environment(left, 'θL')

    q = cirq.LineQubit.range(7)

    circuit = cirq.Circuit.from_ops([
        cirq.H(q[4]),
        cirq.CNOT(q[4], q[5]),
        U1(*q[3:5]),
        U2(*q[2:4]),
        U1(*q[1:3]),
        L(*q[0:2]),
        R(*q[5:7]),
        W(*q[2:5]),
        cirq.inverse(U1_)(*q[1:3]),
        cirq.inverse(U2_)(*q[2:4]),
        cirq.inverse(U1_)(*q[3:5]),
        cirq.CNOT(q[4], q[5]),
        cirq.H(q[4])
        ])

    sim = cirq.Simulator()
    X =sim.simulate(circuit).final_state
    return X 

def scar_obj_fun(params, U1, U2,  W, A_input1, A_input2):
    return np.abs(two_site_sim(params, U1, U2,  W, A_input1, A_input2)[0])*2    


params = np.random.randn(30)

class Neel2(cirq.Gate):
    """
    Gate for the very first step of the many body scar algorithm:
    |-𝕀-|-X-|
    """
    
    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        return [cirq.I(qubits[0]), cirq.X(qubits[1])]
    
    def _circuit_diagram_info_(self, args):
        return 'U1'
    
class Neel1(cirq.Gate):
    """
    Gate for the very first step of the many body scar algorithm:
    |-𝕀-|-𝕀-|
    """

    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        return [cirq.I(qubits[0]), cirq.I(qubits[1])]
    
    def _circuit_diagram_info_(self, args):
        return 'U2'

def make_random_As(noise = False):
    A = np.random.randn(2,2,2)
    
    if noise:
        ζ = 1e-6*np.random.randn(7)
        a1 = [[1,ζ[0]], [ζ[1],ζ[2]]]
        a2 = [[ζ[4],ζ[5]],[ζ[6],ζ[3]]]
    
    if not noise:
        ζ = 1e-7
        a1 = [[1,0], [0,0]]
        a2 = [[0,ζ],[0,0]]

    A1 = A.copy()
    A2 = A.copy()

    A1[0,:,:] = a1
    A1[1,:,:] = a2

    A2[0,:,:] = a2
    A2[1,:,:] = a1
    
    return [A1, A2]

U1 = Neel1()
U2 = Neel2()

As1, As2 = make_random_As()

TimeSteps = 500
dt = 0.01
initial_params = np.random.randn(30)

P = np.array([[1,0],[0,0]])
X = np.array([[0,1],[1,0]])
H = np.kron(np.kron(P,X),P)
W = Tensor(expm(-1j * H * dt), "W")

final_parameters = []

for i in tqdm(range(TimeSteps)):
    if i == 0:
        res = minimize(scar_obj_fun, initial_params, args = (U1, U2, W, As1, As2), method = "Nelder-Mead", options = {'disp':True})
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)
    
    if (i > 0) & (i%2 == 0):
        res = minimize(scar_obj_fun, res.x, args = (U1, U2, W, None, None), method = "Nelder-Mead",options = {'disp':True})
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)

    if (i > 0) & (i%2 == 1):
        # replace the first 15 params with the last 15 and vice versa, so the front 15 params now
        # refer to U2, and the last 15 for U1
        flipped_params = res.x.copy()
        flipped_params[:15] = res.x[15:]
        flipped_params[15:] = res.x[:15]
        res = minimize(scar_obj_fun, flipped_params, args = (U2, U1, W, None, None), method = "Nelder-Mead", options = {'disp':True})
        # Flip the parameters back so the refer to the original direction for the subsequent time step
        temp = res.x.copy()
        res.x[:15] = temp[15:]
        res.x[15:] = temp[:15]
        U1 = ShallowFullStateTensor(2, res.x[:15])
        U2 = ShallowFullStateTensor(2, res.x[15:])
        final_parameters.append(res.x)

np.save('many_body_scar_params', np.array(final_parameters))

        
# -----------------------------------------

# """
# What needs to be done:

# 1) make a multi-site state out of 2 separately parameterized unitary matrices -Done

# 2) figure out a ay to extract an environment from this state  -Done
#         - tensor to unitart,
#         - merge,
#         - get_left/right env
        
# 3) Calculate environment    
    
# 4) get entanglement entropy:
#         - reduced density matrix of the environment 

# 5) Poincare map (have no idea what this is)

# """

Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 721
         Function evaluations: 1123
Optimization terminated successfully.
         Current function value: 0.000006
         Iterations: 941
         Function evaluations: 1406
Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 750
         Function evaluations: 1142
Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 917
         Function evaluations: 1390
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 1105
         Function evaluations: 1659
Optimization terminated successfully.
         Current function value: 0.000003
         Iterations: 687
         Function evaluations: 1108
Optimization terminated successfully.
         Current function value: 0.000011
         Iterations: 701
         Function evaluations: 1114
Optimization

         Current function value: 0.000187
         Iterations: 603
         Function evaluations: 1239
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 591
         Function evaluations: 1089
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 843
         Function evaluations: 1372
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 580
         Function evaluations: 1064
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 864
         Function evaluations: 1408
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 684
         Function evaluations: 1183
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 690
         Function evaluations: 1194
Optimization terminated successfully.
         Curr

         Current function value: 0.000000
         Iterations: 524
         Function evaluations: 1054
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 649
         Function evaluations: 1222
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 547
         Function evaluations: 1106
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 661
         Function evaluations: 1375
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 633
         Function evaluations: 1212
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 628
         Function evaluations: 1165
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 464
         Function evaluations: 1014
Optimization terminated successfully.
         Curr

         Current function value: 0.000000
         Iterations: 587
         Function evaluations: 1175
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1216
         Function evaluations: 1969
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1256
         Function evaluations: 1977
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 802
         Function evaluations: 1453
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 572
         Function evaluations: 1142
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2012
         Function evaluations: 2897
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 700
         Function evaluations: 1338
Optimization terminated successfully.
         C

         Current function value: 0.000000
         Iterations: 548
         Function evaluations: 1125
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 617
         Function evaluations: 1248
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 583
         Function evaluations: 1191
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1526
         Function evaluations: 2363
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 597
         Function evaluations: 1189
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 484
         Function evaluations: 1045
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 562
         Function evaluations: 1143
Optimization terminated successfully.
         Cur

In [67]:
# have parameters that define a U1 and U2.
# build U1 & U2 - ShallowFullStateTensor
# Turn into  As: unitary_to_tensor(cirq.unitary(U))
# Merge A1 and A2
# get_env_exact on merged A12
# Look at the values of the environment to get the entanglement spectrum
params = np.load('many_body_scar_params.npy')
envs = []
for p in params:
    p1 = p[:15]
    p2 = p[15:]
    U1 = ShallowFullStateTensor(2, p1)
    U2 = ShallowFullStateTensor(2, p2)
    
    A1 = unitary_to_tensor(cirq.unitary(U1))
    A2 = unitary_to_tensor(cirq.unitary(U2))
    
    A12 = merge(A1,A2)
    env = get_env_exact(A12)
    
    envs.append(env)


In [68]:
from numpy.linalg import svd
def env_to_entropy(env):
    λ = env[:,0].reshape(2,2)
    _,s,_ = svd(λ)
    # need to take the absolute squares of the values
    s_abs = np.square(np.abs(s))
    Sρ = -np.dot(s_abs, np.log2(s_abs))
    return Sρ


In [69]:
entropies = []
for env in envs:
    entropies.append(env_to_entropy(env))

### Finding Parameterization of the 2 parameter scar mps

In [5]:
A = lambda ϕ, θ: np.array([[[0, 1j*np.exp(-1j*ϕ)], 
                            [0,0]],
                          [[np.cos(θ), 0],
                           [np.sin(θ), 0]]])

U = lambda θ, ϕ: np.array([
    [0,1j*np.exp(-1j*ϕ),0,0],
    [np.cos(θ), 0, 0, -np.sin(θ)],
    [0,0,1,0],
    [np.sin(θ), 0, 0, np.cos(θ)]
])

for x,y in zip(np.random.randn(10), np.random.randn(10)):
    assert np.allclose(U(x,y).conj().T@U(x,y), np.eye(4))

In [146]:
class testAnsatz(cirq.Gate):
    def __init__(self, ϕ, θ):
        self.θ = θ
        self.ϕ = ϕ
        
    def num_qubits(self):
        return 2
    
    def _decompose_(self, qubits):
        q = qubits
        return [
            cirq.ZPowGate()
            cirq.ZPowGate(exponent = (self.ϕ - np.pi/2)/np.pi).on(q[1]),
            cirq.X(q[0]),
            cirq.CNOT(control = q[0],target = q[1]),
            cirq.X(q[0]),
            cirq.CNotPowGate(exponent = 2*self.θ/np.pi).on(control = q[1],target = q[0]),
            cirq.S(q[0])
        ]
    
gate = testAnsatz(0, np.pi/2)

ap(cirq.unitary(gate), _abs = False)

[[0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]]


In [46]:
import numpy as np
from numpy import kron
from functools import reduce

def multi_tensor(Ops):
    return reduce(kron, Ops)

X = np.array([[0,1],[1,0]])
P = np.array([[1,0],[0,0]])
I = np.eye(2)
H = 0.5 * (multi_tensor([P,X,P,I]) + multi_tensor([I,P,X,P]))

In [51]:
from qmps.time_evolve_tools import merge

# bUILDING THE TWO SITES FROM As

def two_site_As(θ, ϕ, θ_, ϕ_):
    A1 = A(ϕ, θ)
    A2 = A(ϕ_, θ_)
    return merge(A1,A2)

def generate_two_site_unitary(params):
    return tensor_to_unitary(two_site_As(*params))

##### Using the Direct Parametrization to build the 2-site unitary:

In [67]:
class ScarGate(cirq.Gate):
    def __init__(self, params):
        self.params = params # this order: [θ, ϕ, θ', ϕ']
        
    def num_qubits(self):
        return 3
    
    def _decompose_(self, qubits):
        q = qubits
        π = np.pi
        θ, ϕ, θ_, ϕ_ = self.params
        return [
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ_ + (π/2))).on(q[2]),
            cirq.X.on(q[1]),
            cirq.CNOT(q[1], q[2]),
            cirq.X.on(q[1]),
            cirq.CNotPowGate(exponent=2*θ_/π).on(q[2], q[1]),
            cirq.S.on(q[1]),
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ + (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CNotPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0])
        ]
    
u_scar = ScarGate([1,2,3,4])

q = cirq.LineQubit.range(3)
c = cirq.Circuit.from_ops([cirq.decompose_once(u_scar.on(*q))])
print(c)
        

0: ───X───────────────────────────────────────────@───X───X──────────S───
                                                  │       │
1: ───X─────────@───X───X───────────S───Z^0.137───X───────@^(7/11)───────
                │       │
2: ───Z^0.773───X───────@^(-1/11)────────────────────────────────────────


##### Trying to get the environment exactly

In [72]:
import sympy as sp
from sympy import symbols,init_printing, I
init_printing(use_latex='mathjax')

θ1, ϕ1, θ2, ϕ2 = symbols('theta_1 phi_1 theta_2 phi_2', real = True)
θ1, ϕ2

A_sp = lambda θ, ϕ: sp.Array([[[0, I*sp.exp(-I*ϕ)], 
                               [0,0]],
                               [[sp.cos(θ), 0],
                               [sp.sin(θ), 0]]])




A1 = A_sp(θ1, ϕ1)
A2 = A_sp(θ2, ϕ2)

from sympy import tensorproduct, tensorcontraction, permutedims

def merge_sp(A1, A2):
    # Merge two Bond Dim 2 Matrix Product States using Sympy
    contracted = tensorcontraction(tensorproduct(A1,A2),(2,4))
    permuted = permutedims(contracted, (0,2,1,3))
    return permuted.reshape(4,2,2)

A12 = merge_sp(A1,A2)
A12 [2,:,:] 

def transfer_matrix_sp(A12):
    # contract two 'merged' objects on their physical index, and group the remaining indices so it is a 4x4 matrix
    flipped = permutedims(A12, (0,1,2))
    flipped = flipped.conjugate()
    contracted = tensorcontraction(tensorproduct(A12, flipped),(0,3))
    grouped = permutedims(contracted,(0,2,1,3))
    return grouped.reshape(4,4).tomatrix()

tm = transfer_matrix_sp(A12)

result = tm.eigenvects()
environment = result[1][2][0].reshape(2,2)
environment

⎡                                        2                                    
⎢                                    -cos (θ₁)                                
⎢                         ────────────────────────────────                    
⎢                            2          2        2                            
⎢                         sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1                    
⎢                                                                             
⎢ ⎛⎛   2          2        2        ⎞                              3        2 
⎢-⎝⎝sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1⎠⋅sin(θ₁)⋅cos(θ₁) - sin(θ₁)⋅cos (θ₁)⋅cos (
⎢─────────────────────────────────────────────────────────────────────────────
⎢                             2          2        2                           
⎣                        - sin (θ₂) - cos (θ₁)⋅cos (θ₂) + 1                   

        ⎛⎛   2          2        2        ⎞                              3    
       -⎝⎝sin (θ₂) + cos (θ₁)⋅cos (θ₂) - 1⎠⋅sin(θ₁)

The analytical expression is not that useful since it appears too complicated to easily express with rotation gates

##### Making the Hamiltonian:

In [99]:
# How to build custom controlled gates:
# - IMPORTANT: LATER VERSIONS OF CIRQ HAVE CHANGED CONTROLLED GATE AND THIS WILL HAVE TO BE CHANGE IF WE UPDATE

q = cirq.LineQubit.range(3)
c = cirq.Circuit.from_ops(
    cirq.ControlledGate(
        sub_gate=cirq.XPowGate(),
        num_controls = 2, 
        control_qubits=[q[0], q[2]]).on(q[1])   
)


In [115]:
class PXPHamiltonian(cirq.Gate):
    def __init__(self, dt):
        self.dt = dt
    
    def num_qubits(self):
        return 4
    
    def _decompose_(self, qubits):
        q = qubits
        return[
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[0], q[2]]).on(q[1]),
            
            cirq.X.on(q[0]), cirq.X.on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3]),

            cirq.ControlledGate(
            sub_gate = cirq.XPowGate(exponent=self.dt / np.pi),
            num_controls = 2,
            control_qubits = [q[1], q[3]]).on(q[2]),
            
            cirq.X.on(q[1]), cirq.X.on(q[3])

        ]
    
    def _circuit_diagram_info_(self, args):
        return ['H']*self.num_qubits()

    
ham = PXPHamiltonian(0.1)

q = cirq.LineQubit.range(4)
c = cirq.Circuit.from_ops([cirq.decompose_once(ham(*q))])
print(c)

0: ───X───@─────────X─────────────────
          │
1: ───────X─────────X───@─────────X───
          │             │
2: ───X───@^0.032───X───X─────────────
                        │
3: ───X─────────────────@^0.032───X───


In [116]:
from xmps.iMPS import Map
from qmps.time_evolve_tools import put_env_on_right_site, put_env_on_left_site
from qmps.represent import Environment 
class ScarGate(cirq.Gate):
    def __init__(self, params):
        self.params = params # this order: [θ, ϕ, θ', ϕ']

    def num_qubits(self):
        return 3

    def _decompose_(self, qubits):
        q = qubits
        π = np.pi
        θ, ϕ, θ_, ϕ_ = self.params
        return [
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ_ + (π/2))).on(q[2]),
            cirq.X.on(q[1]),
            cirq.CNOT(q[1], q[2]),
            cirq.X.on(q[1]),
            cirq.CNotPowGate(exponent=2*θ_/π).on(q[2], q[1]),
            cirq.S.on(q[1]),
            cirq.ZPowGate(exponent=(-1/π) * (-ϕ + (π/2))).on(q[1]),
            cirq.X.on(q[0]),
            cirq.CNOT(q[0], q[1]),
            cirq.X.on(q[0]),
            cirq.CNotPowGate(exponent=2*θ/π).on(q[1], q[0]),
            cirq.S.on(q[0])
        ]
    
    def _circuit_diagram_info_(self, args):
        return ['U']*self.num_qubits()

A = lambda θ, ϕ: np.array([[[0, 1j*np.exp(-1j*ϕ)], 
                            [0,0]],
                           [[np.cos(θ), 0],
                            [np.sin(θ), 0]]])


def scars_time_evolve_cost_function(params, current_params, ham):
    '''
    params are formatted like: [θ1, ϕ1, θ2, ϕ2]
    '''    
    θ1, ϕ1, θ2, ϕ2 = current_params
    θ1_,ϕ1_,θ2_, ϕ2_ = params

    A1 = A(θ1, ϕ1)
    A2 = A(θ2, ϕ2)
    A1_= A(θ1_, ϕ1_)
    A2_= A(θ2_, ϕ2_)
    
    _, r = Map(merge(A1,A2), merge(A1_,A2_)).right_fixed_point()
    R = Environment(put_env_on_left_site(r), 'R')
    L = Environment(put_env_on_right_site(r.conj().T),'L')
    
    U12 = ScarGate(current_params)
    U12_= ScarGate(params)
    q = cirq.LineQubit.range(8)
    circuit = cirq.Circuit.from_ops([
        cirq.H(q[5]),
        cirq.CNOT(q[5],q[6]),
        U12(*q[3:6]),
        U12(*q[1:4]),
        L(*q[0:2]),
        ham(*q[2:6]),
        R(*q[6:8]),
        cirq.inverse(U12_(*q[1:4])),
        cirq.inverse(U12_(*q[3:6])),
        cirq.CNOT(q[5],q[6]),
        cirq.H(q[5])
    ])
    
    print(circuit.to_text_diagram(transpose = True))
    sim = cirq.Simulator()
    ψ = sim.simulate(circuit).final_state[0]
    return np.abs(ψ)*2

scars_time_evolve_cost_function([1,2,3,4], [5,6,7,8], ham)

0 1                                                      2  3                                                      4  5  6 7
│ │                                                      │  │                                                      │  │  │ │
│ │                                                      │  │                                                      │  H  │ │
│ │                                                      │  │                                                      │  │  │ │
│ │                                                      │  │                                                      │  @──X │
│ │                                                      │  │                                                      │  │  │ │
│ │                                                      │  U──────────────────────────────────────────────────────U──U  R─R
│ │                                                      │  │                                                      │  │  │ │


0.8269932866096497